## Intro to Neuroglancer

In [2]:
# - Raw Viewer
import neuroglancer

ip = 'localhost' # or public IP of the machine for sharable display
port = 9999 # change to an unused port number
neuroglancer.set_server_bind_address(bind_address=ip, bind_port=port)

viewer = neuroglancer.Viewer()
print(viewer)

http://localhost:9999/v/ea5eae7de149f0eb10f651da0ea88a28e26fab9c/


In [3]:
# - Example Viewer
import neuroglancer

ip = 'localhost' #or public IP of the machine for sharable display
port = 9999 #change to an unused port number
neuroglancer.set_server_bind_address(bind_address=ip,bind_port=port)

viewer = neuroglancer.Viewer()

with viewer.txn() as s:
    s.layers['image'] = neuroglancer.ImageLayer(source='precomputed://gs://neuroglancer-janelia-flyem-hemibrain/emdata/clahe_yz/jpeg/')
    s.layers['segmentation'] = neuroglancer.SegmentationLayer(source='precomputed://gs://neuroglancer-janelia-flyem-hemibrain/v1.0/segmentation', selected_alpha=0.3)

print(viewer)

http://localhost:9999/v/a95e3d188af11f311b5f7cc402463b813e0c2f43/


In [4]:
# - Using Local Dataset Viewer
# import neuroglancer
# import numpy as np
# import imageio
# import h5py

# ip = 'localhost' #or public IP of the machine for sharable display
# port = 9999 #change to an unused port number
# neuroglancer.set_server_bind_address(bind_address=ip,bind_port=port)
# viewer=neuroglancer.Viewer()

# # SNEMI (# 3d vol dim: z,y,x)
# D0='./'
# res = neuroglancer.CoordinateSpace(
#         names=['z', 'y', 'x'],
#         units=['nm', 'nm', 'nm'],
#         scales=[30, 6, 6])

# print('load im and gt segmentation')
# im = imageio.volread(D0+'train-input.tif')
# with h5py.File(D0+'train_label.h5', 'r') as fl:
#     gt = np.array(fl['main'])
# print(im.shape, gt.shape)

# def ngLayer(data,res,oo=[0,0,0],tt='segmentation'):
#     return neuroglancer.LocalVolume(data,dimensions=res,volume_type=tt,voxel_offset=oo)

# with viewer.txn() as s:
#     s.layers.append(name='im',layer=ngLayer(im,res,tt='image'))
#     s.layers.append(name='gt',layer=ngLayer(gt,res,tt='segmentation'))

# print(viewer)

### Load 36 bit Image on Neuroglancer

In [5]:
from PIL import Image
from PIL import ImageFile
import numpy as np
import os
import neuroglancer
import numpy as np
import imageio
import h5py
from cloudvolume import CloudVolume
from tqdm import tqdm
import gc

In [6]:
# - Read and Show one 36 bit image
Image.MAX_IMAGE_PIXELS = None
ImageFile.LOAD_TRUNCATED_IMAGES = True
raw_image_dir = '/media/mitochondria/Elements/spineheads/raw_images'
segmentation_dir = '/media/mitochondria/Elements/spineheads/segmentations'
segmentation_24_dir = '/media/mitochondria/Elements/spineheads_24'
raw_output_file = '/media/mitochondria/Elements/spineheads/raw_images_h5/all_raw_images.h5'
seg_output_file = '/media/mitochondria/Elements/spineheads/seg_images_h5/all_seg_images.h5'
seg_24_output_file = '/media/mitochondria/Elements/spineheads/seg_images_h5/all_seg_24_images.h5'
split_raw_output_file = './split_raw_h5/split_raw_images.h5'
split_raw_output_dir = './split_raw_h5'
split_seg_output_file = './split_seg_h5/split_seg_images.h5'
split_seg_output_dir = './split_seg_h5'
raw_dataset_name = 'raw_images'
seg_dataset_name = 'seg_images'
n = 10

In [7]:
# - Convert tiff to hd5
def combine_images_to_hdf5(input_dir, output_file, dataset_name, chunk_size=10):
    output_dir = os.path.dirname(output_file)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    image_files = [f for f in sorted(os.listdir(input_dir)) if f.lower().endswith(('png', 'tiff', 'tif'))]
    image_files = image_files
    total_images = len(image_files)
    if total_images == 0:
        print(f"No images found in {input_dir}.")
        return

    # Initialize HDF5 file
    with h5py.File(output_file, 'w') as h5file:
        for chunk_index in tqdm(range(0, total_images, chunk_size), desc="Processing chunks", unit = "file"):
            chunk_files = image_files[chunk_index:chunk_index + chunk_size]
            images = []

            for i in tqdm(range(len(chunk_files)), desc= "Processing files", leave = False):
                file_name = chunk_files[i]
                file_path = os.path.join(input_dir, file_name)
                try:
                    with Image.open(file_path) as img:
                        if file_name.lower().endswith(('tiff', 'tif')) and img.n_frames > 1:
                            print(f"Skipping multi-frame TIFF file: {file_name}")
                            continue
                        img_array = np.array(img)
                        images.append(img_array)
                except Exception as e:
                    print(f"Failed to process image {file_name}: {e}")
                    continue

            if chunk_index == 0:
                # Create dataset with the shape of the first image
                h5file.create_dataset(dataset_name, data=images, maxshape=(None,) + images[0].shape, compression="gzip")
            else:
                # Resize the dataset to accommodate new data
                h5file[dataset_name].resize((h5file[dataset_name].shape[0] + len(images)), axis=0)
                h5file[dataset_name][-len(images):] = images

            del images
            gc.collect()

    print(f"Combined images into {output_file}")

combine_images_to_hdf5(segmentation_24_dir, seg_24_output_file, seg_dataset_name, chunk_size=10)


Processing chunks: 100%|██████████| 3/3 [05:26<00:00, 108.75s/file]

Combined images into /media/mitochondria/Elements/spineheads/seg_images_h5/all_seg_24_images.h5


In [8]:
# - Split 10 Images for Experiments
def split_h5_file(input_file_path, dataset_name, output_file_path, n):
    with h5py.File(input_file_path, 'r') as fl:
        tmp_dataset = fl[dataset_name]
        tmp_array = tmp_dataset[:n]
    with h5py.File(output_file_path, 'w') as fl2:
        fl2.create_dataset(dataset_name, data = tmp_array)


split_h5_file(raw_output_file, raw_dataset_name, split_raw_output_file, 10)
split_h5_file(seg_output_file, seg_dataset_name, split_seg_output_file, 10)


In [9]:
# - Add info file to local dataset
# - Try compress choice???, not sure about all parameters like mesh???
raw_info = CloudVolume.create_new_info(
	num_channels = 1,
	layer_type = 'image', # 'image' or 'segmentation'
	data_type = 'uint8', # can pick any popular uint
	encoding = 'raw', # see: https://github.com/seung-lab/cloud-volume/wiki/Compression-Choices
	resolution = [ 4, 4, 60 ], # X,Y,Z values in nanometers
	voxel_offset = [ 0, 0, 0 ], # values X,Y,Z values in voxels
	chunk_size = [ 1024, 1024, 1 ], # rechunk of image X,Y,Z in voxels
	volume_size = [ 15217, 15260, 10 ], # X,Y,Z size in voxels
)

raw_vol = CloudVolume(f"file://{split_raw_output_dir}", info = raw_info)
raw_vol.commit_info()

seg_info = CloudVolume.create_new_info(
	num_channels = 1,
	layer_type = 'segmentation', # 'image' or 'segmentation'
	data_type = 'uint32', # can pick any popular uint
	encoding = 'raw', # see: https://github.com/seung-lab/cloud-volume/wiki/Compression-Choices
	resolution = [ 4, 4, 60 ], # X,Y,Z values in nanometers
	voxel_offset = [ 0, 0, 0 ], # values X,Y,Z values in voxels
	chunk_size = [ 1024, 1024, 1 ], # rechunk of image X,Y,Z in voxels
	volume_size = [ 15217, 15260, 10 ], # X,Y,Z size in voxels
    mesh = 'mesh'
)

seg_vol = CloudVolume(f"file://{split_seg_output_dir}", info = raw_info)
seg_vol.commit_info()


In [10]:
# - Check raw image bit size unit8 grayscale
raw_image_sample = Image.open(raw_image_dir + '/raw_image_000.png')
print(raw_image_sample.mode)

L


In [11]:
def choose_first_n_images(file_path, dataset_name, n):
    with h5py.File(file_path, 'r') as fl:
        tmp_dataset = fl[dataset_name]
        images = []
        for i in range(n):
            images.append(tmp_dataset[i])
        # if dataset_name == 'raw_images':
        #     return np.array(images)
        # else:
        #     return np.array(images, dtype=np.uint32)
        if dataset_name == 'seg_images':
            return np.array(images, dtype=np.uint32)
        return np.array(images)


def neuroglancer_visualize(raw_output_file, seg_output_file):
    ip = 'localhost'
    port = 9999
    neuroglancer.set_server_bind_address(bind_address=ip, bind_port=port)
    viewer = neuroglancer.Viewer()

    res = neuroglancer.CoordinateSpace(
        names=['z', 'y', 'x'],
        units=['nm', 'nm', 'nm'],
        scales=[60, 4, 4]
    )

    print('Load raw image and segmentation.')
    try:
        im = choose_first_n_images(raw_output_file, 'raw_images', n)
        gt = choose_first_n_images(seg_output_file, 'seg_images', n)
    except FileNotFoundError as e:
        print(e)
        return

    print(im.shape, gt.shape)

    def ngLayer(data, res, oo=[0, 0, 0], tt='segmentation'):
        return neuroglancer.LocalVolume(data, dimensions=res, volume_type=tt, voxel_offset=oo)

    with viewer.txn() as s:
        s.layers.append(name='im', layer=ngLayer(im, res, tt='image'))
        s.layers.append(name='gt', layer=ngLayer(gt, res, tt='segmentation'))
        # all_unique_ids = np.unique(gt)
        # for id in all_unique_ids:
        #     s.layers['segmentation'].segments.add(id)

    print(viewer)

In [12]:
with h5py.File(seg_24_output_file, 'r') as fl:
    tmp_dataset = fl[seg_dataset_name]
    images = []
    for i in range(n):
        images.append(tmp_dataset[i])
    seg_24_dataset = np.array(images, dtype=np.uint32)

In [17]:
all_unique_channels = []
for i in range(3):
    all_unique_channels.append(np.unique(seg_24_dataset[:, :, :, i]))
    

In [31]:
np.sum(seg_24_dataset[:, :, :, 1] != seg_24_dataset[:, :, :, 0])

240685280

In [13]:
neuroglancer_visualize(raw_output_file, seg_24_output_file)

Load raw image and segmentation.
(10, 15260, 15217) (10, 15260, 15217, 3)


ValueError: rank of data (4) must match rank of coordinate space (3)

PS: You could also directly use the GUI of the web-based application to load diverse datasets.

In [ ]:
!pip install tifffile

### Neuroglancer Tutorial

In [ ]:
# - Experiment with 32 bit SNEMI
import os
from PIL import Image
from tifffile import *
import numpy as np
D0='./SNEMI_example/'
# raw_input_path = D0+'train-input.tif'
label_input_path = D0+'train-labels.tif'

label_output_path = D0 + 'train-lables-32bit.tif'

image_stack = imread(label_input_path)
image_stack_uint32 = image_stack.astype(np.uint32)
imwrite(label_output_path, image_stack, photometric='minisblack')


In [ ]:
with Image.open(label_input_path) as img:
    print(img.mode)

I;16B


In [ ]:
image_stack.dtype

dtype('<u2')

In [ ]:
# - Experiment with SNEMI
import neuroglancer
import numpy as np
import imageio
import h5py

ip = 'localhost' #or public IP of the machine for sharable display
port = 9999 #change to an unused port number
neuroglancer.set_server_bind_address(bind_address=ip,bind_port=port)
viewer=neuroglancer.Viewer()

# SNEMI (# 3d vol dim: z,y,x)
D0='./SNEMI_example/'
res = neuroglancer.CoordinateSpace(
        names=['z', 'y', 'x'],
        units=['nm', 'nm', 'nm'],
        scales=[30, 6, 6])

print('load im and gt segmentation')
im = imageio.volread(D0+'train-input.tif')
gt = imageio.volread(D0+'train-lables.tif')
print(im.shape, gt.shape)

def ngLayer(data,res,oo=[0,0,0],tt='segmentation'):
    return neuroglancer.LocalVolume(data,dimensions=res,volume_type=tt,voxel_offset=oo)

with viewer.txn() as s:
    s.layers.append(name='im',layer=ngLayer(im,res,tt='image'))
    s.layers.append(name='gt',layer=ngLayer(gt,res,tt='segmentation'))

print(viewer)

load im and gt segmentation


FileNotFoundError: No such file: '/home/mitochondria/Desktop/yuxuan_exp/neuroglancer_tools/SNEMI_example/train-lables.tif'